In [ ]:
pip install datasets

     |████████████████████████████████| 264 kB 12.7 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 118 kB 58.1 MB/s 
     |████████████████████████████████| 243 kB 58.8 MB/s 


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train-output-types.csv',
                                          'test': 'test-output-types.csv'})

Using custom data configuration default-ad58f3c0fef90e34


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ad58f3c0fef90e34/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'code', 'label'],
        num_rows: 1283
    })
    test: Dataset({
        features: ['text', 'code', 'label'],
        num_rows: 254
    })
})


In [ ]:
pip install transformers

     |████████████████████████████████| 2.6 MB 14.9 MB/s 
     |████████████████████████████████| 636 kB 48.2 MB/s 
     |████████████████████████████████| 3.3 MB 32.4 MB/s 
     |████████████████████████████████| 895 kB 32.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.15
    Uninstalling huggingface-hub-0.0.15:
      Successfully uninstalled huggingface-hub-0.0.15


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import torch
print(torch.__version__)

1.9.0+cu102


In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=10)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments("test_trainer")

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [ ]:
dataset["train"][0]

{'code': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))',
 'label': 0,
 'text': "Concatenate elements of a list 'x' of multiple integers to a single integer"}

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running training *****
  Num examples = 1283
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 483


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=483, training_loss=0.746808670075537, metrics={'train_runtime': 14892.2811, 'train_samples_per_second': 0.258, 'train_steps_per_second': 0.032, 'total_flos': 1012787194116096.0, 'train_loss': 0.746808670075537, 'epoch': 3.0})

In [ ]:
import numpy as np

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("accuracy")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running Evaluation *****
  Num examples = 254
  Batch size = 8


{'eval_accuracy': 0.6732283464566929,
 'eval_loss': 1.1609586477279663,
 'eval_runtime': 313.3069,
 'eval_samples_per_second': 0.811,
 'eval_steps_per_second': 0.102}

In [ ]:
trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running Prediction *****
  Num examples = 254
  Batch size = 8


PredictionOutput(predictions=array([[-0.57843506,  6.1279416 , -1.3433841 , ..., -0.63539344,
        -0.9801806 ,  0.10528993],
       [ 0.24672976, -1.034431  , -0.19538777, ..., -0.7233115 ,
        -0.4024375 , -0.23333558],
       [-0.1283288 ,  5.838603  , -1.2045842 , ..., -0.6747505 ,
        -1.2686048 ,  0.13179058],
       ...,
       [ 0.13218477,  0.7628559 , -1.2738141 , ..., -0.43051133,
        -0.5473717 , -1.3255746 ],
       [-0.28684413, -0.59747416, -0.4356827 , ..., -0.45692596,
        -0.5360513 , -1.7840028 ],
       [ 3.8624833 , -1.3417376 , -0.7195095 , ..., -0.24384834,
        -0.2681002 , -1.8424271 ]], dtype=float32), label_ids=array([1, 6, 1, 2, 2, 1, 9, 5, 5, 1, 1, 9, 0, 0, 5, 2, 5, 1, 0, 4, 1, 1,
       5, 1, 1, 1, 1, 9, 0, 5, 1, 2, 2, 5, 5, 4, 4, 1, 1, 5, 5, 1, 5, 5,
       5, 5, 1, 5, 9, 5, 1, 1, 5, 1, 5, 1, 1, 1, 0, 2, 5, 2, 0, 0, 0, 0,
       4, 5, 1, 5, 7, 5, 5, 5, 1, 0, 5, 5, 2, 5, 5, 5, 5, 1, 1, 1, 1, 5,
       5, 6, 5, 5, 5, 4, 4, 4, 1, 4, 5, 

In [ ]:
test_trainer = Trainer(model) 

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
raw_pred, _, _ = test_trainer.predict(test_dataset=tokenized_datasets['test']) 

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running Prediction *****
  Num examples = 254
  Batch size = 8


In [ ]:
y_pred = np.argmax(raw_pred, axis=1)
print(y_pred)

[1 6 1 5 5 5 1 5 5 5 5 0 0 0 5 2 5 1 0 5 5 1 5 1 1 1 9 9 5 5 1 2 2 5 5 5 5
 5 1 5 5 0 5 0 5 5 6 5 9 0 1 1 5 1 5 1 1 1 0 0 0 0 0 0 0 0 2 5 0 5 5 5 5 5
 1 1 5 5 2 5 5 5 1 1 1 1 5 5 5 6 5 5 5 5 5 5 1 5 5 6 5 5 5 1 1 5 5 5 0 0 0
 5 2 5 2 5 1 5 0 2 6 5 1 1 1 0 5 5 5 5 1 5 0 0 0 0 0 0 0 5 8 1 2 5 7 5 5 5
 5 1 5 5 2 2 2 2 0 5 5 5 5 5 1 5 5 5 1 5 2 5 2 0 6 6 6 6 6 5 5 5 5 0 1 5 5
 5 2 1 5 0 0 0 0 0 0 0 1 5 5 5 5 5 2 5 5 5 0 0 0 0 0 0 0 5 1 5 5 1 1 1 1 5
 0 0 5 1 5 0 5 5 5 5 4 5 5 1 5 5 9 1 0 5 5 6 2 1 5 5 5 9 0 5 5 0]
